# Overfitting Experimentation

In model_selection.ipynb we selected convolutional neural networks as our model class and identified overfitting as a major issue with this dataset. 
In this notebook we experiment with different ways to configure our model for reducing overfitting and with including more variety into our data by using data generators.

We do this in two independent experiments. 




. These are:
- Dropout Layers
- Batch Normalization
- Regularization
- weight constraints

In addition, we experiment with data generators to include more variety in our dataset.




In this notebook we carry out experiments to select which machine learning model 

- feature engineering
- proportional train test split
- data Generator
- dropout
- batch normalization
- statistical tests

In [1]:
# correct working directory. But only once. 
if not "working_directory_corrected" in vars():
    %cd ..
    working_directory_corrected = True

import numpy as np
import matplotlib.pyplot as plt 

import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential  # Model type to be used
from tensorflow.keras.layers import Dense, Activation # Types of layers to be used in our model
from tensorflow.keras.layers import Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, ZeroPadding2D, GlobalAveragePooling2D, Flatten

from models.CNN_Builder import CNN_Builder

from evaluation.multi_run_evaluation import Multi_Run_Evaluation
from evaluation.evaluation_metrics import Evaluation_Metrics
from data.dataset import Dataset
dataset = Dataset()

c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project


## Data Loading

Before carrying out the experiments, let's load our dataset.

In addition to loading our dataset we will use image generators for the remainder of the experiment. For now, we will generate dummy image generators that don't include any changes.

In [2]:
X,y = dataset.get_prepared_data()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
y_train_prob = np.array([y_train[i] for i in range(len(y_train))])

train_datagen = ImageDataGenerator(horizontal_flip=False,
                                   vertical_flip=False,
                                   fill_mode="nearest")
train_generator = train_datagen.flow(X_train,y_train_prob)

## Experiment 1: Network Configuration

In this experiment we measure the influence of network configurations on generalization. For this purpose we test the following configurations:
- Dropout Layers
- Batch Normalization
- Regularization
- weight constraints

### Experiment Setup:
For our experiment we learn the network identified in previous experiments once with each of the configuration parameters and without and compare the hamming score.

As we have noted in past experiments, the variance of different weight initializations is a concern. For this reason, we implemented the class *Multi_Run_Evaluaton* which can run an experiment multiple times, recording the evaluation metrics and calculating minimum, maximum, mean and standard deviation. 

To make the configuration code less verbous and less error prone we also impemented a builder lass that assembles the CNN. 

During our experiment we ran the cell below with different values for *apply_regularization*, *apply_dropout*, *apply_batch_normalization* and *weight_constraints" and recorded results in the cell below. 

In [3]:


cnn_builder = CNN_Builder(convolutional_layers=[16],
                            fully_connected_layers=[100],
                            in_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3]),
                            out_shape=y_train.shape[1])
cnn_builder.apply_regularization = True
cnn_builder.apply_dropout = True
cnn_builder.apply_batch_normalization = True
cnn_builder.weight_constraints = True



evaluator = Multi_Run_Evaluation(cnn_builder.build_model)

evaluator.evaluate( nr_runs=10, 
                    epochs=100, 
                    early_stopping_patience=5, 
                    train_generator= train_generator, 
                    X_train=X_train, 
                    y_train=y_train, 
                    X_test=X_test,
                    y_test=y_test)


evaluator.print_metrics()





 running experiment 1 of 10


c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


Epoch 1/100
21/21 [==============================] - 13s 597ms/step - loss: 2.1565 - val_loss: 2.2298
Epoch 2/100
21/21 [==============================] - 13s 608ms/step - loss: 2.1450 - val_loss: 2.1809
Epoch 3/100
21/21 [==============================] - 12s 582ms/step - loss: 2.0888 - val_loss: 2.1179
Epoch 4/100
21/21 [==============================] - 12s 582ms/step - loss: 2.0279 - val_loss: 2.0662
Epoch 5/100
21/21 [==============================] - 11s 515ms/step - loss: 1.9513 - val_loss: 1.9886
Epoch 6/100
21/21 [==============================] - 11s 538ms/step - loss: 1.8771 - val_loss: 1.9363
Epoch 7/100
21/21 [==============================] - 11s 531ms/step - loss: 1.8197 - val_loss: 1.8648
Epoch 8/100
21/21 [==============================] - 11s 529ms/step - loss: 1.7747 - val_loss: 1.8164
Epoch 9/100
21/21 [==============================] - 11s 527ms/step - loss: 1.7477 - val_loss: 1.7790
Epoch 10/100
21/21 [==============================] - 11s 530ms/step - loss: 1.739

c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


21/21 [==============================] - 12s 586ms/step - loss: 2.1760 - val_loss: 2.4142
Epoch 2/100
21/21 [==============================] - 14s 656ms/step - loss: 2.1579 - val_loss: 2.3192
Epoch 3/100
21/21 [==============================] - 17s 785ms/step - loss: 2.0926 - val_loss: 2.2832
Epoch 4/100
21/21 [==============================] - 12s 589ms/step - loss: 2.0480 - val_loss: 2.2240
Epoch 5/100
21/21 [==============================] - 12s 568ms/step - loss: 1.9784 - val_loss: 2.1070
Epoch 6/100
21/21 [==============================] - 12s 584ms/step - loss: 1.8982 - val_loss: 2.0198
Epoch 7/100
21/21 [==============================] - 15s 718ms/step - loss: 1.8344 - val_loss: 1.9273
Epoch 8/100
21/21 [==============================] - 12s 582ms/step - loss: 1.7957 - val_loss: 1.8477
Epoch 9/100
21/21 [==============================] - 11s 540ms/step - loss: 1.7638 - val_loss: 1.8060
Epoch 10/100
21/21 [==============================] - 12s 557ms/step - loss: 1.7486 - val_loss

...

21/21 [==============================] - 13s 629ms/step - loss: 2.1438 - val_loss: 2.3859
Epoch 2/100
21/21 [==============================] - 13s 644ms/step - loss: 2.1381 - val_loss: 2.3879
Epoch 3/100
21/21 [==============================] - 11s 533ms/step - loss: 2.1011 - val_loss: 2.3446
Epoch 4/100
21/21 [==============================] - 11s 504ms/step - loss: 2.0359 - val_loss: 2.2968
Epoch 5/100
21/21 [==============================] - 11s 507ms/step - loss: 1.9619 - val_loss: 2.2308
Epoch 6/100
21/21 [==============================] - 12s 572ms/step - loss: 1.8922 - val_loss: 2.1922
Epoch 7/100
21/21 [==============================] - 11s 505ms/step - loss: 1.8234 - val_loss: 2.0668
Epoch 8/100
21/21 [==============================] - 11s 506ms/step - loss: 1.7932 - val_loss: 2.0233
Epoch 9/100
21/21 [==============================] - 11s 512ms/step - loss: 1.7622 - val_loss: 1.9452
Epoch 10/100
21/21 [==============================] - 11s 547ms/step - loss: 1.7459 - val_loss

c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


21/21 [==============================] - 11s 519ms/step - loss: 2.1557 - val_loss: 2.2178
Epoch 2/100
21/21 [==============================] - 11s 515ms/step - loss: 2.1389 - val_loss: 2.1560
Epoch 3/100
21/21 [==============================] - 11s 512ms/step - loss: 2.0890 - val_loss: 2.1572
Epoch 4/100
21/21 [==============================] - 12s 575ms/step - loss: 2.0410 - val_loss: 2.0747
Epoch 5/100
21/21 [==============================] - 13s 612ms/step - loss: 1.9619 - val_loss: 1.9616
Epoch 6/100
21/21 [==============================] - 11s 535ms/step - loss: 1.8840 - val_loss: 1.8683
Epoch 7/100
21/21 [==============================] - 11s 535ms/step - loss: 1.8204 - val_loss: 1.8053
Epoch 8/100
21/21 [==============================] - 11s 540ms/step - loss: 1.7810 - val_loss: 1.7687
Epoch 9/100
21/21 [==============================] - 13s 607ms/step - loss: 1.7525 - val_loss: 1.7585
Epoch 10/100
21/21 [==============================] - 12s 568ms/step - loss: 1.7399 - val_loss

c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


21/21 [==============================] - 11s 517ms/step - loss: 2.1435 - val_loss: 2.3745
Epoch 2/100
21/21 [==============================] - 11s 506ms/step - loss: 2.1472 - val_loss: 2.3288
Epoch 3/100
21/21 [==============================] - 11s 507ms/step - loss: 2.0853 - val_loss: 2.3451
Epoch 4/100
21/21 [==============================] - 11s 510ms/step - loss: 2.0195 - val_loss: 2.3039
Epoch 5/100
21/21 [==============================] - 11s 515ms/step - loss: 1.9513 - val_loss: 2.2830
Epoch 6/100
21/21 [==============================] - 12s 581ms/step - loss: 1.8657 - val_loss: 2.1947
Epoch 7/100
21/21 [==============================] - 14s 659ms/step - loss: 1.8096 - val_loss: 2.0981
Epoch 8/100
21/21 [==============================] - 12s 547ms/step - loss: 1.7699 - val_loss: 2.0213
Epoch 9/100
21/21 [==============================] - 11s 529ms/step - loss: 1.7461 - val_loss: 1.9715
Epoch 10/100
21/21 [==============================] - 12s 573ms/step - loss: 1.7343 - val_loss

### Results

Running our Experiment yielded the following results:

| Configuration          | Min  | Max  | Mean  | St. Dev |
| ---------------------- | ---- | ---- | ----- | ------- |
| No Configuration       | 0.17 | 0.21 | 0.195 | 0.012   |
| Regularization         | 0.16 | 0.21 | 0.184 | 0.017   |
| Dropout                | 0.17 | 0.21 | 0.19  | 0.011   |
| Batch Normalization    | 0.02 | 0.16 | 0.109 | 0.040   |
| Weight Constraints     | 0.11 | 0.20 | 0.163 | 0.029   |

### Interpretation
None of the configurations have improved the Hamming score of the test set. While regularization and Dropout are close - and thus could be a result of random variation, batch normalization led to a significantly worse performance. 

This does not necessarily mean, that these configurations are not useful. It may mean that their effect is overshadowed by the general issue of having a very small data set. In previous experiments we have seen dropout increase the hammilton score slightly, so there seems to be some promise. 

For now, this experiment is inconclusive. we will revisit this experiment after increasing the data set complexity with data generators, hoping that this will increase the complexity enough for these regularizations to make a consistent difference. 



## Experiment 2: Image Generators

In this experiment we introduce variabilty into the training set by using image generators. We hope that this increases the variability of input seen by the model and helps it generalize. 

Specifically, we try to:
- Mirror images horizonzally
- Rotate the image up to 30 degrees in either direction
- Shift the image by 10% of the image size in horizontal or vertical direction
- change brightness by 20%
- zooming in or out by 20%

### Experiment Setup:
For our experiment we learn the network identified in previous experiments once with each of the configuration parameters and without and compare the hamming score.

As in the previoius experiment, we use the CNN Factory and Multi Run Evaluation.
During our experiment we ran the cell below with different convigurations of the train data generator and recorded results in the cell below. 

In [4]:

train_datagen = ImageDataGenerator(
                                    #rotation_range=30,
                                    #zoom_range = 0.20,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    # brightness_range= [0.8, 1.2],
                                    #horizontal_flip=True,
                                   fill_mode="nearest")
train_generator = train_datagen.flow(X_train,y_train_prob)

cnn_builder = CNN_Builder(convolutional_layers=[16],
                            fully_connected_layers=[100],
                            in_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3]),
                            out_shape=y_train.shape[1])


evaluator = Multi_Run_Evaluation(cnn_builder.build_model)

evaluator.evaluate( nr_runs=10, 
                    epochs=100, 
                    early_stopping_patience=5, 
                    train_generator= train_generator, 
                    X_train=X_train, 
                    y_train=y_train, 
                    X_test=X_test,
                    y_test=y_test)


evaluator.print_metrics()


 running experiment 1 of 10


c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


Epoch 1/100
21/21 [==============================] - 10s 458ms/step - loss: 0.3169 - val_loss: 0.2781
Epoch 2/100
21/21 [==============================] - 10s 446ms/step - loss: 0.2790 - val_loss: 0.2784
Epoch 3/100
21/21 [==============================] - 9s 445ms/step - loss: 0.2783 - val_loss: 0.2771
Epoch 4/100
21/21 [==============================] - 9s 444ms/step - loss: 0.2756 - val_loss: 0.2756
Epoch 5/100
21/21 [==============================] - 9s 439ms/step - loss: 0.2719 - val_loss: 0.2717
Epoch 6/100
21/21 [==============================] - 9s 444ms/step - loss: 0.2674 - val_loss: 0.2707
Epoch 7/100
21/21 [==============================] - 9s 443ms/step - loss: 0.2638 - val_loss: 0.2687
Epoch 8/100
21/21 [==============================] - 10s 447ms/step - loss: 0.2578 - val_loss: 0.2664
Epoch 9/100
21/21 [==============================] - 10s 442ms/step - loss: 0.2547 - val_loss: 0.2671
Epoch 10/100
21/21 [==============================] - 10s 444ms/step - loss: 0.2525 - v

c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


Epoch 1/100
21/21 [==============================] - 9s 458ms/step - loss: 0.2998 - val_loss: 0.2767
Epoch 2/100
21/21 [==============================] - 10s 446ms/step - loss: 0.2757 - val_loss: 0.2753
Epoch 3/100
21/21 [==============================] - 10s 442ms/step - loss: 0.2708 - val_loss: 0.2724
Epoch 4/100
21/21 [==============================] - 9s 442ms/step - loss: 0.2664 - val_loss: 0.2697
Epoch 5/100
21/21 [==============================] - 10s 447ms/step - loss: 0.2637 - val_loss: 0.2694
Epoch 6/100
21/21 [==============================] - 9s 452ms/step - loss: 0.2590 - val_loss: 0.2704
Epoch 7/100
21/21 [==============================] - 10s 457ms/step - loss: 0.2559 - val_loss: 0.2702
Epoch 8/100
21/21 [==============================] - 10s 452ms/step - loss: 0.2521 - val_loss: 0.2676
Epoch 9/100
21/21 [==============================] - 11s 521ms/step - loss: 0.2512 - val_loss: 0.2725
Epoch 10/100
21/21 [==============================] - 9s 455ms/step - loss: 0.2508 - 

...

Epoch 1/100
21/21 [==============================] - 10s 457ms/step - loss: 0.3166 - val_loss: 0.2799
Epoch 2/100
21/21 [==============================] - 10s 449ms/step - loss: 0.2785 - val_loss: 0.2788
Epoch 3/100
21/21 [==============================] - 10s 455ms/step - loss: 0.2722 - val_loss: 0.2742
Epoch 4/100
21/21 [==============================] - 10s 448ms/step - loss: 0.2684 - val_loss: 0.2717
Epoch 5/100
21/21 [==============================] - 10s 447ms/step - loss: 0.2646 - val_loss: 0.2734
Epoch 6/100
21/21 [==============================] - 9s 444ms/step - loss: 0.2641 - val_loss: 0.2741
Epoch 7/100
21/21 [==============================] - 10s 451ms/step - loss: 0.2606 - val_loss: 0.2724
Epoch 8/100
21/21 [==============================] - 10s 462ms/step - loss: 0.2585 - val_loss: 0.2720
Epoch 9/100
21/21 [==============================] - 10s 471ms/step - loss: 0.2555 - val_loss: 0.2680
Epoch 10/100
21/21 [==============================] - 10s 478ms/step - loss: 0.2530

c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


Epoch 1/100
21/21 [==============================] - 10s 451ms/step - loss: 0.3223 - val_loss: 0.2806
Epoch 2/100
21/21 [==============================] - 10s 447ms/step - loss: 0.2760 - val_loss: 0.2766
Epoch 3/100
21/21 [==============================] - 10s 457ms/step - loss: 0.2722 - val_loss: 0.2734
Epoch 4/100
21/21 [==============================] - 10s 449ms/step - loss: 0.2695 - val_loss: 0.2705
Epoch 5/100
21/21 [==============================] - 9s 442ms/step - loss: 0.2667 - val_loss: 0.2712
Epoch 6/100
21/21 [==============================] - 9s 445ms/step - loss: 0.2643 - val_loss: 0.2726
Epoch 7/100
21/21 [==============================] - 9s 442ms/step - loss: 0.2597 - val_loss: 0.2703
Epoch 8/100
21/21 [==============================] - 10s 447ms/step - loss: 0.2590 - val_loss: 0.2679
Epoch 9/100
21/21 [==============================] - 10s 447ms/step - loss: 0.2552 - val_loss: 0.2652
Epoch 10/100
21/21 [==============================] - 10s 444ms/step - loss: 0.2519 -

c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


Epoch 1/100
21/21 [==============================] - 10s 452ms/step - loss: 0.3149 - val_loss: 0.2782
Epoch 2/100
21/21 [==============================] - 9s 444ms/step - loss: 0.2758 - val_loss: 0.2746
Epoch 3/100
21/21 [==============================] - 9s 441ms/step - loss: 0.2708 - val_loss: 0.2703
Epoch 4/100
21/21 [==============================] - 9s 439ms/step - loss: 0.2671 - val_loss: 0.2716
Epoch 5/100
21/21 [==============================] - 9s 443ms/step - loss: 0.2650 - val_loss: 0.2682
Epoch 6/100
21/21 [==============================] - 9s 440ms/step - loss: 0.2614 - val_loss: 0.2687
Epoch 7/100
21/21 [==============================] - 10s 456ms/step - loss: 0.2584 - val_loss: 0.2664
Epoch 8/100
21/21 [==============================] - 11s 500ms/step - loss: 0.2538 - val_loss: 0.2677
Epoch 9/100
21/21 [==============================] - 9s 455ms/step - loss: 0.2520 - val_loss: 0.2653
Epoch 10/100
21/21 [==============================] - 10s 446ms/step - loss: 0.2508 - va

### Results

Running our Experiment yielded the following results:

| Configuration          | Min  | Max  | Mean  | St. Dev |
| ---------------------- | ---- | ---- | ----- | ------- |
| No Configuration       | 0.17 | 0.21 | 0.195 | 0.012   |
| Mirroring              | 0.17 | 0.22 | 0.191 | 0.015   |
| Rotation               | 0.19 | 0.24 | 0.214 | 0.014   |
| Shift                  | 0.21 | 0.25 | 0.231 | 0.015   |
| Brightness             | 0.08 | 0.13 | 0.111 | 0.019   |
| Zoom                   | 0.17 | 0.22 | 0.198 | 0.018   |

### Interpretation

Our results indicate that both rotation and shift seem to increase the hamming score and changing brightness dramatically decrased the score. Mirroring and zoom did not result in significant score changes.

Based on these results we identified the following follow-up questions:
* How do the results change if we activate all configurations except brightness?
* How do the results change if we activate only rotation and shift?
* Is the score increase from shift consistent? 

To answer these question the following additional data was recorded:

| Configuration          | Min  | Max  | Mean  | St. Dev |
| ---------------------- | ---- | ---- | ----- | ------- |
| No Configuration       | 0.17 | 0.21 | 0.195 | 0.012   |
| All except brightness  | 0.19 | 0.25 | 0.214 | 0.017   |
| Rotation + Shift       | 0.18 | 0.24 | 0.218 | 0.022   |
| Shift (rerun)          | 0.18 | 0.24 | 0.222 | 0.018 |

From these results we can see that activating all configurations except brightness yields the same quality as activating only brightness or only shift. The same holds for only activating rotation and shift. The effects of these preprocessing techniques do not seem to confound each other - or at least this is not visible in the relatively simple model we applied.

Rerunning sift also yielded a slighty lower result than before. For the future we decided to use rotation and shift for data preprocessing. 

As a side-node: our rerun of the shift preprocessing also shows that we should be careful even in interpreting the averaged results over ten runs. We will take this into account when doing bigger experiments by using more runs.


## Experiment 3: Increasing complexity
In this experiment we test whether we can improve our scores by increasing the network complexity. 

### Experiment Setup
For our experiment we use the configuration of the image data generator that performed best in Experiment 2 and test different complexity increases. We test:
- Adding convolutional layers
- increasing the number of convolutional patterns
- increasing the number of fully connected neurons
- adding more fully connected neurons. 

Due to technical difficulties, we had to restart the notebook environment for this test. This means, data recorded here is done with a different trainin / test split than the other experiments. The experiment itself has been executed entirely with the same training test split.

As in the previoius experiment, we use the CNN Factory and Multi Run Evaluation.
During our experiment we ran the cell below with different convigurations of the train data generator and recorded results in the cell below. 

In [5]:

train_datagen = ImageDataGenerator(
                                    rotation_range=30,
                                    #zoom_range = 0.20,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    # brightness_range= [0.8, 1.2],
                                    #horizontal_flip=True,
                                   fill_mode="nearest")
train_generator = train_datagen.flow(X_train,y_train_prob)

cnn_builder = CNN_Builder(convolutional_layers=[64, 32,16],
                            fully_connected_layers=[100],
                            in_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3]),
                            out_shape=y_train.shape[1])


evaluator = Multi_Run_Evaluation(cnn_builder.build_model)

evaluator.evaluate( nr_runs=10, 
                    epochs=100, 
                    early_stopping_patience=5, 
                    train_generator= train_generator, 
                    X_train=X_train, 
                    y_train=y_train, 
                    X_test=X_test,
                    y_test=y_test)


evaluator.print_metrics()


 running experiment 1 of 10


c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


Epoch 1/100
21/21 [==============================] - 23s 1s/step - loss: 0.3385 - val_loss: 0.2932
Epoch 2/100
21/21 [==============================] - 23s 1s/step - loss: 0.2842 - val_loss: 0.2803
Epoch 3/100
21/21 [==============================] - 22s 1s/step - loss: 0.2814 - val_loss: 0.2795
Epoch 4/100
21/21 [==============================] - 23s 1s/step - loss: 0.2790 - val_loss: 0.2782
Epoch 5/100
21/21 [==============================] - 22s 1s/step - loss: 0.2800 - val_loss: 0.2791
Epoch 6/100
21/21 [==============================] - 22s 1s/step - loss: 0.2784 - val_loss: 0.2780
Epoch 7/100
21/21 [==============================] - 23s 1s/step - loss: 0.2766 - val_loss: 0.2778
Epoch 8/100
21/21 [==============================] - 22s 1s/step - loss: 0.2750 - val_loss: 0.2772
Epoch 9/100
21/21 [==============================] - 22s 1s/step - loss: 0.2731 - val_loss: 0.2752
Epoch 10/100
21/21 [==============================] - 22s 1s/step - loss: 0.2708 - val_loss: 0.2728
Epoch 11/

c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


Epoch 1/100
21/21 [==============================] - 23s 1s/step - loss: 0.3409 - val_loss: 0.2840
Epoch 2/100
21/21 [==============================] - 22s 1s/step - loss: 0.2841 - val_loss: 0.2812
Epoch 3/100
21/21 [==============================] - 23s 1s/step - loss: 0.2821 - val_loss: 0.2790
Epoch 4/100
21/21 [==============================] - 23s 1s/step - loss: 0.2807 - val_loss: 0.2806
Epoch 5/100
21/21 [==============================] - 22s 1s/step - loss: 0.2801 - val_loss: 0.2821
Epoch 6/100
21/21 [==============================] - 23s 1s/step - loss: 0.2784 - val_loss: 0.2792
Epoch 7/100
21/21 [==============================] - 22s 1s/step - loss: 0.2782 - val_loss: 0.2789
Epoch 8/100
21/21 [==============================] - 22s 1s/step - loss: 0.2786 - val_loss: 0.2785
Epoch 9/100
21/21 [==============================] - 23s 1s/step - loss: 0.2770 - val_loss: 0.2756
Epoch 10/100
21/21 [==============================] - 22s 1s/step - loss: 0.2770 - val_loss: 0.2791
Epoch 11/

...

Epoch 1/100
21/21 [==============================] - 28s 1s/step - loss: 0.3299 - val_loss: 0.2815
Epoch 2/100
21/21 [==============================] - 29s 1s/step - loss: 0.2826 - val_loss: 0.2793
Epoch 3/100
21/21 [==============================] - 29s 1s/step - loss: 0.2799 - val_loss: 0.2792
Epoch 4/100
21/21 [==============================] - 26s 1s/step - loss: 0.2789 - val_loss: 0.2773
Epoch 5/100
21/21 [==============================] - 26s 1s/step - loss: 0.2780 - val_loss: 0.2770
Epoch 6/100
21/21 [==============================] - 26s 1s/step - loss: 0.2778 - val_loss: 0.2768
Epoch 7/100
21/21 [==============================] - 26s 1s/step - loss: 0.2741 - val_loss: 0.2803
Epoch 8/100
21/21 [==============================] - 25s 1s/step - loss: 0.2724 - val_loss: 0.2779
Epoch 9/100
21/21 [==============================] - 26s 1s/step - loss: 0.2705 - val_loss: 0.2764
Epoch 10/100
21/21 [==============================] - 26s 1s/step - loss: 0.2700 - val_loss: 0.2760
Epoch 11/

c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


Epoch 1/100
21/21 [==============================] - 26s 1s/step - loss: 0.3418 - val_loss: 0.2832
Epoch 2/100
21/21 [==============================] - 25s 1s/step - loss: 0.2824 - val_loss: 0.2814
Epoch 3/100
21/21 [==============================] - 22s 1s/step - loss: 0.2818 - val_loss: 0.2821
Epoch 4/100
21/21 [==============================] - 22s 1s/step - loss: 0.2807 - val_loss: 0.2821
Epoch 5/100
21/21 [==============================] - 23s 1s/step - loss: 0.2787 - val_loss: 0.2779
Epoch 6/100
21/21 [==============================] - 24s 1s/step - loss: 0.2751 - val_loss: 0.2796
Epoch 7/100
21/21 [==============================] - 28s 1s/step - loss: 0.2716 - val_loss: 0.2769
Epoch 8/100
21/21 [==============================] - 23s 1s/step - loss: 0.2725 - val_loss: 0.2720
Epoch 9/100
21/21 [==============================] - 24s 1s/step - loss: 0.2708 - val_loss: 0.2732
Epoch 10/100
21/21 [==============================] - 24s 1s/step - loss: 0.2676 - val_loss: 0.2751
Epoch 11/

c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


Epoch 1/100
21/21 [==============================] - 24s 1s/step - loss: 0.3409 - val_loss: 0.2862
Epoch 2/100
21/21 [==============================] - 26s 1s/step - loss: 0.2844 - val_loss: 0.2825
Epoch 3/100
21/21 [==============================] - 26s 1s/step - loss: 0.2822 - val_loss: 0.2793
Epoch 4/100
21/21 [==============================] - 29s 1s/step - loss: 0.2810 - val_loss: 0.2790
Epoch 5/100
21/21 [==============================] - 27s 1s/step - loss: 0.2805 - val_loss: 0.2817
Epoch 6/100
21/21 [==============================] - 24s 1s/step - loss: 0.2816 - val_loss: 0.2791
Epoch 7/100
21/21 [==============================] - 31s 1s/step - loss: 0.2810 - val_loss: 0.2781
Epoch 8/100
21/21 [==============================] - 31s 1s/step - loss: 0.2796 - val_loss: 0.2774
Epoch 9/100
21/21 [==============================] - 26s 1s/step - loss: 0.2780 - val_loss: 0.2780
Epoch 10/100
21/21 [==============================] - 25s 1s/step - loss: 0.2784 - val_loss: 0.2778
Epoch 11/

### Results


Running our Experiment yielded the following results:

| Test | Convolutional Layers | Fully Connected Layers | Min  | Max  | Mean  | St. Dev |
| ---- | -------------------- | ---------------------- | ---- | ----- | ---- | ------- |
| 1    | [16]                 | [100]                  | 0.20 | 0.23 | 0.217 | 0.012   |
| 2    | [32]                 | [100]                  | 0.16 | 0.22 | 0.201 | 0.021   |
| 3    | [32,16]              | [100]                  | 0.20 | 0.23 | 0.215 | 0.013   |
| 4    | [16]                 | [1000]                 | 0.15 | 0.23 | 0.204 | 0.023   |
| 5    | [16]                 | [100,100]              | 0.15 | 0.24 | 0.213 | 0.025   |
| 6    | [32,16]              | [100,100]              | 0.15 | 0.24 | 0.19  | 0.031   |
| 7    | [32,16]              | [20,20]                | 0.09 | 0.22 | 0.175 | 0.038   |
| 8    | [64,32,16]           | [100,100]              | 0.11 | 0.15 | 0.125 | 0.014   |
| 9    | [64,32,16]           | [100]                  | 0.10 | 0.23 | 0.173 | 0.03    |


### Interpretation

The numbers indicate that increasing the complexity did not improve results and in some cases significantly worsened the results. The best configurations we found were configurations 1, 3 and 5.
We will use these as starting points for future fine-tuning.



# Experiment 4: Network configuration in complex networks

In this experiment we rerun experiment 1 for a more complex network, to see if this changes the results and indicates that we should use any of the configurations.

## Experiment Setup: 

For experimentation we used configuration 6 from the previous experiment. While this configuration yielded worse scores than others, it is a combination of the two promising configurations 3 and 5. We did not use either of these two directly as we deem them to simple for this experiment. 


As in the previoius experiment, we use the CNN Factory and Multi Run Evaluation.
During our experiment we ran the cell below with different convigurations of the train data generator and recorded results in the cell below. 

In [6]:

train_datagen = ImageDataGenerator(
                                    rotation_range=30,
                                    #zoom_range = 0.20,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    # brightness_range= [0.8, 1.2],
                                    #horizontal_flip=True,
                                   fill_mode="nearest")
train_generator = train_datagen.flow(X_train,y_train_prob)

cnn_builder = CNN_Builder(convolutional_layers=[32,16],
                            fully_connected_layers=[100,100],
                            in_shape=(X_train.shape[1],X_train.shape[2],X_train.shape[3]),
                            out_shape=y_train.shape[1])
cnn_builder.apply_regularization = False
cnn_builder.apply_dropout = False
cnn_builder.apply_batch_normalization = False
cnn_builder.weight_constraints = False


evaluator = Multi_Run_Evaluation(cnn_builder.build_model)

evaluator.evaluate( nr_runs=10, 
                    epochs=100, 
                    early_stopping_patience=5, 
                    train_generator= train_generator, 
                    X_train=X_train, 
                    y_train=y_train, 
                    X_test=X_test,
                    y_test=y_test)


evaluator.print_metrics()


 running experiment 1 of 10


c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


Epoch 1/100
21/21 [==============================] - 15s 710ms/step - loss: 0.3271 - val_loss: 0.2811
Epoch 2/100
21/21 [==============================] - 15s 696ms/step - loss: 0.2830 - val_loss: 0.2830
Epoch 3/100
21/21 [==============================] - 14s 674ms/step - loss: 0.2818 - val_loss: 0.2796
Epoch 4/100
21/21 [==============================] - 14s 683ms/step - loss: 0.2814 - val_loss: 0.2799
Epoch 5/100
21/21 [==============================] - 15s 717ms/step - loss: 0.2802 - val_loss: 0.2808
Epoch 6/100
21/21 [==============================] - 17s 804ms/step - loss: 0.2783 - val_loss: 0.2786
Epoch 7/100
21/21 [==============================] - 16s 754ms/step - loss: 0.2759 - val_loss: 0.2742
Epoch 8/100
21/21 [==============================] - 16s 746ms/step - loss: 0.2733 - val_loss: 0.2739
Epoch 9/100
21/21 [==============================] - 19s 914ms/step - loss: 0.2732 - val_loss: 0.2760
Epoch 10/100
21/21 [==============================] - 15s 717ms/step - loss: 0.269

c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


Epoch 1/100
21/21 [==============================] - 15s 725ms/step - loss: 0.3271 - val_loss: 0.2811
Epoch 2/100
21/21 [==============================] - 17s 788ms/step - loss: 0.2836 - val_loss: 0.2815
Epoch 3/100
21/21 [==============================] - 15s 716ms/step - loss: 0.2825 - val_loss: 0.2823
Epoch 4/100
21/21 [==============================] - 15s 719ms/step - loss: 0.2815 - val_loss: 0.2766
Epoch 5/100
21/21 [==============================] - 15s 702ms/step - loss: 0.2808 - val_loss: 0.2750
Epoch 6/100
21/21 [==============================] - 15s 725ms/step - loss: 0.2777 - val_loss: 0.2783
Epoch 7/100
21/21 [==============================] - 21s 973ms/step - loss: 0.2765 - val_loss: 0.2776
Epoch 8/100
21/21 [==============================] - 19s 888ms/step - loss: 0.2749 - val_loss: 0.2719
Epoch 9/100
21/21 [==============================] - 17s 792ms/step - loss: 0.2728 - val_loss: 0.2711
Epoch 10/100
21/21 [==============================] - 17s 786ms/step - loss: 0.269

...

Epoch 1/100
21/21 [==============================] - 19s 915ms/step - loss: 0.3336 - val_loss: 0.2815
Epoch 2/100
21/21 [==============================] - 18s 842ms/step - loss: 0.2823 - val_loss: 0.2813
Epoch 3/100
21/21 [==============================] - 18s 843ms/step - loss: 0.2818 - val_loss: 0.2789
Epoch 4/100
21/21 [==============================] - 18s 835ms/step - loss: 0.2828 - val_loss: 0.2795
Epoch 5/100
21/21 [==============================] - 18s 860ms/step - loss: 0.2815 - val_loss: 0.2780
Epoch 6/100
21/21 [==============================] - 18s 852ms/step - loss: 0.2806 - val_loss: 0.2792
Epoch 7/100
21/21 [==============================] - 18s 837ms/step - loss: 0.2808 - val_loss: 0.2781
Epoch 8/100
21/21 [==============================] - 17s 816ms/step - loss: 0.2807 - val_loss: 0.2795
Epoch 9/100
21/21 [==============================] - 17s 809ms/step - loss: 0.2795 - val_loss: 0.2776
Epoch 10/100
21/21 [==============================] - 18s 834ms/step - loss: 0.280

c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


Epoch 1/100
21/21 [==============================] - 18s 852ms/step - loss: 0.3420 - val_loss: 0.2820
Epoch 2/100
21/21 [==============================] - 19s 894ms/step - loss: 0.2820 - val_loss: 0.2797
Epoch 3/100
21/21 [==============================] - 18s 854ms/step - loss: 0.2806 - val_loss: 0.2824
Epoch 4/100
21/21 [==============================] - 18s 868ms/step - loss: 0.2812 - val_loss: 0.2794
Epoch 5/100
21/21 [==============================] - 19s 870ms/step - loss: 0.2802 - val_loss: 0.2780
Epoch 6/100
21/21 [==============================] - 17s 809ms/step - loss: 0.2779 - val_loss: 0.2761
Epoch 7/100
21/21 [==============================] - 16s 736ms/step - loss: 0.2733 - val_loss: 0.2747
Epoch 8/100
21/21 [==============================] - 20s 931ms/step - loss: 0.2711 - val_loss: 0.2728
Epoch 9/100
21/21 [==============================] - 16s 756ms/step - loss: 0.2695 - val_loss: 0.2728
Epoch 10/100
21/21 [==============================] - 16s 738ms/step - loss: 0.267

c:\Users\frank\Documents\Teaching\LU\Spring2023\Machine Learning² Unit\Machine Learning Example Project\evaluation\multi_run_evaluation.py:49: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping])


Epoch 1/100
21/21 [==============================] - 15s 710ms/step - loss: 0.3263 - val_loss: 0.2829
Epoch 2/100
21/21 [==============================] - 15s 683ms/step - loss: 0.2826 - val_loss: 0.2807
Epoch 3/100
21/21 [==============================] - 15s 736ms/step - loss: 0.2807 - val_loss: 0.2827
Epoch 4/100
21/21 [==============================] - 15s 725ms/step - loss: 0.2816 - val_loss: 0.2791
Epoch 5/100
21/21 [==============================] - 14s 688ms/step - loss: 0.2806 - val_loss: 0.2804
Epoch 6/100
21/21 [==============================] - 16s 734ms/step - loss: 0.2782 - val_loss: 0.2787
Epoch 7/100
21/21 [==============================] - 13s 600ms/step - loss: 0.2767 - val_loss: 0.2762
Epoch 8/100
21/21 [==============================] - 15s 711ms/step - loss: 0.2745 - val_loss: 0.2743
Epoch 9/100
21/21 [==============================] - 14s 657ms/step - loss: 0.2724 - val_loss: 0.2732
Epoch 10/100
21/21 [==============================] - 14s 676ms/step - loss: 0.268

### Results

Running our Experiment yielded the following results:

| Configuration          | Min  | Max  | Mean  | St. Dev |
| ---------------------- | ---- | ---- | ----- | ------- |
| No Configuration (again!)      | 0.12 | 0.24 | 0.194 | 0.032   | 
| Regularization         | 0.13 | 0.19 | 0.175 | 0.018   |
| Dropout                | 0.1  | 0.23 | 0.177 | 0.047   |
| Batch Normalization    | 0.07 | 0.16 | 0.117 | 0.026   |
| Weight Constraints     | 0.16 | 0.21 | 0.184 | 0.016   |


### Interpretation.

The results achieved mirror those of Experiment 1. The configurations we tested seem to not make a big difference in changing the hamming score. Batch normalization in particular worsens the score significantly.

In the future, we will use regularization, dropout and weight constraints. Although we will do so more out of recognition of good practices as this experiment did not give cause to assume that they will improve matters drastically.

## Conclusions

The experiments performed in this notebook showed that we can improve the generalization of our models by using image generators to increase the variety of input images. The neural network configuration did not improve results but also did not worsen them.

As an outcome of the experiments, we derive the following knowledge for further experiments:
- Experiment 1 and 4: We will use regularization, dropount and weight constraints for our models.
- Experiment 2: We will use image generators with rotation and shift for image data preprocessing.
- Experiment 3: We will focus on models in the low-complexity end of CNNs. This means  we will test ...
    - no more than 2 convolutional layers
    - no more than 1000 Neurons in fully connected layers
    - no more than 2 fully connected layers.

We will use these for future experiments.

In addition, we noted that weven with 10 runs, the numbers can differ significantly when rerunning an experiment. For this reason, we will use 100 runs in the future. 